# Economic Data Analysis with Fred & Pandas

In [ ]:
!pip install fredapi > /dev/null

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

plt.style.use('fivethirtyeight')
pd.set_option('display.max_columns', 500)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]

from fredapi import Fred

fred_key = 'e13a36efb5e28d626f5f0241662f0007'

# Create the Fred Object

In [ ]:
fred = Fred(api_key=fred_key)

# Search for economic data

In [ ]:
sp_search = fred.search('S&P',order_by='popularity')

In [ ]:
sp_search.head()

# Pull Raw Data & Plot

In [ ]:
sp500 = fred.get_series(series_id='SP500')

In [ ]:
sp500.plot(figsize=(10,5), title='S&P 500', lw=2)

# Pull and Join Multiple Data Series

In [ ]:
unemployment_df = fred.search('unemployment rate state', filter=('frequency', 'Monthly'))
unemployment_df = unemployment_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')
unemployment_df = unemployment_df.loc[unemployment_df['title'].str.contains('Unemployment Rate in')]
unemployment_df = unemployment_df.loc[unemployment_df['id'].str.contains('R')]
unemployment_df = unemployment_df.loc[~unemployment_df['title'].str.contains('Census')]

 

In [ ]:
all_results = []

for myid in unemployment_df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    all_results.append(results)

unemploymnet_states = pd.concat(all_results, axis=1)
id_to_state = unemployment_df['title'].str.replace('Unemployment Rate in','').to_dict()
unemploymnet_states.columns = [id_to_state[c] for c in unemploymnet_states.columns]

In [ ]:
unemploymnet_states = unemploymnet_states.dropna()
unemploymnet_states = unemploymnet_states.drop(' Puerto Rico', axis=1)

In [ ]:
# Plot States Unemploymnet Rate
px.line(unemploymnet_states)

In [ ]:
ax = unemploymnet_states.loc[unemploymnet_states.index == '2020-05-01'].T.sort_values('2020-05-01').plot(kind='barh', figsize=(10, 13), title='Unemploymnet by State, April 2020')

ax.legend().remove()

# Pull Participation Rate

In [ ]:
part_df = fred.search('participation rate state', filter=('frequency', 'Monthly'))
part_df = part_df.query('seasonal_adjustment == "Seasonally Adjusted" and units == "Percent"')
part_df = part_df.loc[part_df['title'].str.contains('Labor Force Participation Rate for')]

In [ ]:
part_id_to_state = part_df['title'].str.replace('Labor Force Participation Rate for', '').to_dict()

all_results = []

for myid in part_df.index:
    results = fred.get_series(myid)
    results = results.to_frame(name=myid)
    all_results.append(results)
part_states = pd.concat(all_results, axis=1)
part_states.columns = [part_id_to_state[c] for c in part_states.columns]


# Plot Unemployment vs Participation

In [ ]:
# Fix DC
unemploymnet_states = unemploymnet_states.rename(columns={' the District of Columbia': ' District of Columbia'})

In [ ]:
fig, axs = plt.subplots(10,5, figsize=(30,30), sharex=True)
axs = axs.flatten()

i = 0
for state in unemploymnet_states.columns:
    if state == ' District of Columbia':
        continue
    ax2 = axs[i].twinx()
    part_states.query('index >= 2020 and index < 2023')[state].plot(ax=ax2, label="Participation")
    unemploymnet_states.query('index >= 2020 and index < 2023')[state].plot(ax=axs[i], label="Unemployment", color=color_pal[1])
    ax2.grid(False)
    axs[i].set_title(state)
    i += 1
plt.tight_layout()
plt.show()